In [19]:
import numpy as np
import pandas as pd

In [24]:
pd.Timestamp(year=2,month=10,day=20) + pd.DateOffset(months=3)

Timestamp('3-01-20 00:00:00')

In [2]:
users = pd.read_csv("data/Users.csv")
useractivity = pd.read_csv("data/UserActivity.csv")
compsp = pd.read_csv("data/CompetitionPartipation.csv")
comps = pd.read_csv("data/Competition.csv")
blogs = pd.read_csv("data/Blogs.csv")
comments = pd.read_csv("data/Comments.csv")
discussion = pd.read_csv("data/Discussion.csv")
jobs = pd.read_csv("data/Jobs.csv")

In [3]:
original_months = {2:7, 3:8, 4:9, 5:10, 6:11, 7:12, 8:1, 9:2, 10:3, 11:4, 12:5, 1:6}

In [4]:
users['Created At Month'] = users['Created At Month'].map(original_months)

In [ ]:
pd.to_datetime()

In [16]:
users['Creation_date'] = users.apply(lambda x : pd.Timestamp(f"{x['Created At Month']}-{x['Created At Day_of_month']}"),axis=1) 

In [17]:
users

,FeatureX,FeatureY,User_ID,Countries_ID,Created At time,Created At Year,Created At Month,Created At Day_of_month,Creation_date
0,0,0,ID_DC6S4E9O,X9GR,19:33:13.663391,1,6,16,0001-06-16 00:00:00
1,1,0,ID_E8S97OUT,X9GR,22:18:18.228921,1,5,27,0001-05-27 00:00:00
2,0,0,ID_QZ1HASL3,X9GR,23:13:03.266635,1,6,1,0001-06-01 00:00:00
3,1,0,ID_SDV9Q3E9,X9GR,09:50:09.697573,1,5,27,0001-05-27 00:00:00
4,1,0,ID_W942LN7F,X9GR,13:33:06.531662,1,6,16,0001-06-16 00:00:00
...,...,...,...,...,...,...,...,...,...
12408,0,1,ID_C5393PD0,7DKX,05:02:39.352427,1,4,6,0001-04-06 00:00:00
12409,0,1,ID_QX7XRT0C,K4DG,02:50:19.027320,1,8,25,0001-08-25 00:00:00
12410,0,1,ID_RGN2RK66,ONON,22:06:33.713692,1,5,7,0001-05-07 00:00:00
12411,0,0,ID_XZS1QE0W,3YQR,06:56:02.570173,1,10,27,0001-10-27 00:00:00


In [ ]:
submission_count_keys = compsp['Successful Submission Count'].unique()
submission_count_keys = submission_count_keys[~pd.isna(submission_count_keys)]
def get_comp_per_user(row):
    account_creation_day = int(row['Created At Day_of_month'])
    account_creation_month = int(row['Created At Month'])
    nextmonth = (account_creation_month + 1) % 12
    nextday = (account_creation_day + 1) % 31
    competitions_per_user = compsp.loc[(row['User_ID'] == compsp['User_ID']) 
                                       & (((compsp['Created At Month'] == account_creation_month) 
                                           & (compsp['Created At Day_of_month'] >= account_creation_day)) 
                                           |((compsp['Created At Month'] == nextmonth) 
                                           & (compsp['Created At Day_of_month'] < nextday)))]
    submission_counts = competitions_per_user['Successful Submission Count'].value_counts()
    submission_counts = [int(submission_counts.get(key,0)) for key in submission_count_keys]
    return pd.Series([len(competitions_per_user)]+submission_counts)

def get_comments_per_user(row):
    account_creation_month = row['Created At Month']
    comments_per_user = comments.loc[(row['User_ID'] == comments['User_ID']) & (comments['Created At Month'] == account_creation_month) ]
    return len(comments_per_user)

def get_discussion_per_user(row):
    account_creation_month = row['Created At Month']
    discussions_per_user = discussion.loc[(row['User_ID'] == discussion['User_ID']) & (discussion['Created At Month'] == account_creation_month) ]
    return len(discussions_per_user)

def split_time(row):
    time = pd.Timestamp(row['Created At time'])
    hour = int(time.hour)
    minutes = int(time.minute)
    seconds = int(time.second)
    return pd.Series([hour,minutes,seconds])


In [289]:
jobs_activity = useractivity[useractivity['Title'].str.startswith('job')]
unique_jobs = jobs_activity.drop_duplicates(subset=['Title','User_ID'])
job_count = unique_jobs.groupby('User_ID')['Title'].nunique()
job_count = pd.Series(job_count)
users['job_count'] = users['User_ID'].map(job_count).fillna(0).astype('int')

In [290]:
blogs_activity = useractivity[useractivity['Title'].str.startswith('job')]
unique_blogs = blogs_activity.drop_duplicates(subset=['Title','User_ID'])
blog_count = unique_blogs.groupby('User_ID')['Title'].nunique()
blog_count = pd.Series(blog_count)
users['blog_activity_count'] = users['User_ID'].map(blog_count).fillna(0).astype('int')

In [291]:
submission_count_keys = list(map(lambda x : "subm "+ x,submission_count_keys))
users[['competitons_count'] + submission_count_keys] = users.apply(get_comp_per_user,axis=1)
users['comments_count'] = users.apply(get_comments_per_user,axis=1)
users['discussions_count'] = users.apply(get_discussion_per_user,axis=1)
users[['hour','minute','second']] = users.apply(split_time,axis=1)

In [351]:
useractivitymerged = useractivity.merge(users[['User_ID','Created At Month','Created At Day_of_month']],on='User_ID',how='left')

useractivitymergedfiltered = useractivitymerged.loc[((useractivitymerged['datetime Month'] == useractivitymerged['Created At Month'])
                                                    & (useractivitymerged['datetime Day_of_month'] >= useractivitymerged['Created At Day_of_month'])) 
                                                    | ((useractivitymerged['datetime Month'] == (useractivitymerged['Created At Month'] + 1)%12)
                                                    & (useractivitymerged['datetime Day_of_month'] < (useractivitymerged['Created At Day_of_month']+1)%31))]

users['activity_days_count'] = users['User_ID'].map(useractivitymergedfiltered['User_ID'].value_counts()).fillna(0).astype('int')

In [352]:
useractivitymergedfilteredtarget = useractivitymerged.loc[((useractivitymerged['datetime Month'] == (useractivitymerged['Created At Month']+1)%12)
                                                    & (useractivitymerged['datetime Day_of_month'] >= (useractivitymerged['Created At Day_of_month']+1)%31)) 
                                                    | ((useractivitymerged['datetime Month'] == (useractivitymerged['Created At Month'] + 2)%12)
                                                    & (useractivitymerged['datetime Day_of_month'] < (useractivitymerged['Created At Day_of_month']+2)%31))]


In [335]:
users

,FeatureX,FeatureY,User_ID,Countries_ID,Created At time,Created At Year,Created At Month,Created At Day_of_month,job_count,blog_activity_count,...,subm count 6,subm count 7,subm count 5,subm count 4,comments_count,discussions_count,hour,minute,second,activity_days_count
0,0,0,ID_DC6S4E9O,X9GR,19:33:13.663391,1,1,16,0,0,...,0,0,0,0,0,0,19,33,13,11
1,1,0,ID_E8S97OUT,X9GR,22:18:18.228921,1,12,27,2,2,...,0,0,0,0,0,0,22,18,18,184
2,0,0,ID_QZ1HASL3,X9GR,23:13:03.266635,1,1,1,0,0,...,0,0,0,0,0,0,23,13,3,21
3,1,0,ID_SDV9Q3E9,X9GR,09:50:09.697573,1,12,27,0,0,...,0,0,0,0,0,0,9,50,9,105
4,1,0,ID_W942LN7F,X9GR,13:33:06.531662,1,1,16,0,0,...,0,0,0,0,2,0,13,33,6,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12408,0,1,ID_C5393PD0,7DKX,05:02:39.352427,1,11,6,0,0,...,0,0,0,0,0,0,5,2,39,0
12409,0,1,ID_QX7XRT0C,K4DG,02:50:19.027320,1,3,25,0,0,...,0,0,0,0,0,0,2,50,19,12
12410,0,1,ID_RGN2RK66,ONON,22:06:33.713692,1,12,7,0,0,...,0,0,0,0,0,0,22,6,33,26
12411,0,0,ID_XZS1QE0W,3YQR,06:56:02.570173,1,5,27,0,0,...,0,0,0,0,0,0,6,56,2,9


In [353]:
useractivitymergedfilteredtarget = users['User_ID'].map(useractivitymergedfilteredtarget['User_ID'].value_counts() > 0).fillna(0).astype('int')

In [366]:
useractivitymergedfilteredtarget.sum()

np.int64(1800)

In [338]:
discussion

,Competition ID,Personal,Theme,Disc_ID,User_ID,Created At time,Created At Year,Created At Month,Created At Day_of_month
0,GeneralDiscussion,1,NaN,ID_0BK7TH,ID_AAGFVLHH,19:48:26.862175,1,12,27
1,GeneralDiscussion,1,NaN,ID_ZFJ9J7,ID_0AJGQILU,09:37:52.039526,1,12,20
2,GeneralDiscussion,1,NaN,ID_ME45XX,ID_U90KK6KL,17:50:33.894800,1,2,9
3,ID_45W9,1,NaN,ID_2BG9IX,ID_U90KK6KL,07:57:55.178845,1,2,10
4,GeneralDiscussion,1,NaN,ID_251XVV,ID_F7HIGFHQ,19:56:13.041893,1,12,27
...,...,...,...,...,...,...,...,...,...
1434,GeneralDiscussion,1,NaN,ID_09W0TZ,ID_AAJKI0P3,14:15:31.081484,1,5,23
1435,GeneralDiscussion,1,NaN,ID_613AUJ,ID_8A3X5IWU,11:06:11.201934,1,5,14
1436,GeneralDiscussion,1,NaN,ID_TCN8TB,ID_3NL0QLE6,04:57:31.578153,1,5,12
1437,GeneralDiscussion,1,NaN,ID_FTJ37Z,ID_3PL1LQQX,05:11:59.313759,1,5,9


In [339]:
discussionmerged = discussion.merge(users[['User_ID','Created At Month','Created At Day_of_month']],on='User_ID',how='left')


In [340]:
discussionmerged

,Competition ID,Personal,Theme,Disc_ID,User_ID,Created At time,Created At Year,Created At Month_x,Created At Day_of_month_x,Created At Month_y,Created At Day_of_month_y
0,GeneralDiscussion,1,NaN,ID_0BK7TH,ID_AAGFVLHH,19:48:26.862175,1,12,27,12,20
1,GeneralDiscussion,1,NaN,ID_ZFJ9J7,ID_0AJGQILU,09:37:52.039526,1,12,20,12,20
2,GeneralDiscussion,1,NaN,ID_ME45XX,ID_U90KK6KL,17:50:33.894800,1,2,9,2,8
3,ID_45W9,1,NaN,ID_2BG9IX,ID_U90KK6KL,07:57:55.178845,1,2,10,2,8
4,GeneralDiscussion,1,NaN,ID_251XVV,ID_F7HIGFHQ,19:56:13.041893,1,12,27,12,27
...,...,...,...,...,...,...,...,...,...,...,...
1434,GeneralDiscussion,1,NaN,ID_09W0TZ,ID_AAJKI0P3,14:15:31.081484,1,5,23,5,16
1435,GeneralDiscussion,1,NaN,ID_613AUJ,ID_8A3X5IWU,11:06:11.201934,1,5,14,5,11
1436,GeneralDiscussion,1,NaN,ID_TCN8TB,ID_3NL0QLE6,04:57:31.578153,1,5,12,5,6
1437,GeneralDiscussion,1,NaN,ID_FTJ37Z,ID_3PL1LQQX,05:11:59.313759,1,5,9,5,9


In [345]:
discussionmergedfilteredtarget = discussionmerged.loc[((discussionmerged['Created At Month_x'] == (discussionmerged['Created At Month_y']+1)%12)
                                                    & (discussionmerged['Created At Day_of_month_x'] >= (discussionmerged['Created At Day_of_month_y']+1)%31)) 
                                                    | ((discussionmerged['Created At Month_x'] == (discussionmerged['Created At Month_y'] + 2)%12)
                                                    & (discussionmerged['Created At Day_of_month_x'] < (discussionmerged['Created At Day_of_month_y']+2)%31))]


In [346]:
discussionmergedfilteredtarget['User_ID'].value_counts() > 0

User_ID
ID_CEDNKSVV    True
ID_6FWILK4H    True
ID_0HI8E3M8    True
ID_AQIBV924    True
ID_PNADKFXR    True
               ... 
ID_0W7CRJ5S    True
ID_WZ6WNU2T    True
ID_EPAX9S0O    True
ID_5FD3I6FB    True
ID_EKFB430Z    True
Name: count, Length: 92, dtype: bool

In [347]:
discussionmergedfilteredtarget = users['User_ID'].map(discussionmergedfilteredtarget['User_ID'].value_counts() > 0).fillna(0).astype('int')

In [363]:
discussionmergedfilteredtarget.sum()

np.int64(92)

In [354]:
commentsmerged = comments.merge(users[['User_ID','Created At Month','Created At Day_of_month']],on='User_ID',how='left')
commentsmergedfilteredtarget = commentsmerged.loc[((commentsmerged['Created At Month_x'] == (commentsmerged['Created At Month_y']+1)%12)
                                                    & (commentsmerged['Created At Day_of_month_x'] >= (commentsmerged['Created At Day_of_month_y']+1)%31)) 
                                                    | ((commentsmerged['Created At Month_x'] == (commentsmerged['Created At Month_y'] + 2)%12)
                                                    & (commentsmerged['Created At Day_of_month_x'] < (commentsmerged['Created At Day_of_month_y']+2)%31))]
commentsmergedfilteredtarget = users['User_ID'].map(commentsmergedfilteredtarget['User_ID'].value_counts() > 0).fillna(0).astype('int')

In [364]:
commentsmergedfilteredtarget.sum()

np.int64(36)

In [360]:
target = (commentsmergedfilteredtarget) | (discussionmergedfilteredtarget) | (useractivitymergedfilteredtarget)

In [361]:
target.sum()

np.int64(1815)

In [367]:
users['target'] = target

In [369]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12413 entries, 0 to 12412
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   FeatureX                 12413 non-null  int64 
 1   FeatureY                 12413 non-null  int64 
 2   User_ID                  12413 non-null  object
 3   Countries_ID             6531 non-null   object
 4   Created At time          12413 non-null  object
 5   Created At Year          12413 non-null  int64 
 6   Created At Month         12413 non-null  int64 
 7   Created At Day_of_month  12413 non-null  int64 
 8   job_count                12413 non-null  int64 
 9   blog_activity_count      12413 non-null  int64 
 10  competitons_count        12413 non-null  int64 
 11  subm count 10            12413 non-null  int64 
 12  subm count 3             12413 non-null  int64 
 13  subm count 8             12413 non-null  int64 
 14  subm count 9             12413 non-nul